### Import required packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from pyod.models.knn import KNN
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from math import sqrt
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification


In [ ]:
# Data from Ethiopia
Births_Ethiopia = pd.read_sas('Births_Ethiopia.SAS7BDAT')
print(Births_Ethiopia.shape)

# 'M19': Birth_Weight: Birth weight in kilograms (3 decimals) [Original Response variable]
print("The number of valid samples in the dataset is " + str(len(Births_Ethiopia['M19'].dropna())))
                                                            
Births_Ethiopia.head()

## Original 17 features
'V026': Residence: Type of place of residence (Large city/ Small city/ Town/ Countryside)

'V106': Education: Educational level (no education/primary/secondary or higher)

'H42': Iron: Taking iron pills, sprinkles or syrup

'V190': Wealth: Wealth index combined (Poorest/poorer/middle/richer/richest)

'V445': BMI: Body mass index (numerical)

'V447A': Age_W: Women's age in years (numerical)

'V457': Anemia: Anemia level (severe/ moderate /mild/not anemic)

'V463A': Smoking: Smokes cigarettes (no/yes)

'V481': Insurance: Covered by health insurance (no/yes)

'V717': Occupation: Respondent's occupation

'V131': Ethnicity: Ethnicity (Amahara/Oromo/Tigrie/Somali/Guragie/Others

'V501': Marital: Current marital status

'V130': Religion: Religion(orthodox/muslim/protestant/others)

'B4': Gender: Sex of child (male/female)

'B0': Twin: The child is a twin (single/multiple)

'M14': Visits: Number of antenatal visits during pregnancy (numerical)

'M17': Delivery_Caesarean: Delivery by caesarean section (no/yes)

In [ ]:
Births_Ethiopia_df = Births_Ethiopia[['M19','V026','V106','H42','V190','V445','V447A','V457','V463A','V481','V717','V131',
                  'V501','V130','B4','B0','M14','M17']]
Births_Ethiopia_df.columns = ['Birth_Weight','Residence','Education','Iron','Wealth','BMI','Age_W','Anemia',
                     'Smoking','Insurance','Occupation','Ethnicity','Marital','Religion','Gender',
                    'Twin','Visits','Delivery_Caesarean']
Births_Ethiopia_df = Births_Ethiopia_df.dropna(subset=['Birth_Weight'])
Births_Ethiopia_df = Births_Ethiopia_df.reset_index()
Births_Ethiopia_df = Births_Ethiopia_df.iloc[:,1:]
print(Births_Ethiopia_df.shape)
Births_Ethiopia_df.head()

## Data Pre-processing

In [ ]:
# Categorical variables: one-hot encoding
enc = OneHotEncoder(sparse=False)
encoded_array = enc.fit_transform(Births_Ethiopia_df[['Residence', 'Education', 'Iron', 'Wealth', 'Anemia', 'Smoking', 'Insurance', 'Occupation', 'Ethnicity', 'Marital', 'Religion', 'Gender', 'Twin', 'Delivery_Caesarean']])

encoded_cols = enc.get_feature_names_out(['Residence', 'Education', 'Iron', 'Wealth', 'Anemia', 'Smoking', 'Insurance', 'Occupation', 'Ethnicity', 'Marital', 'Religion', 'Gender', 'Twin', 'Delivery_Caesarean'])
encoded_df = pd.DataFrame(encoded_array, columns=encoded_cols)

# Drop the original columns that were encoded & concatenate with the other columns
Births_Ethiopia_encoded = pd.concat([Births_Ethiopia_df.drop(['Residence', 'Education', 'Iron', 'Wealth', 'Anemia', 'Smoking', 'Insurance', 'Occupation', 'Ethnicity', 'Marital', 'Religion', 'Gender', 'Twin', 'Delivery_Caesarean'], axis=1), encoded_df], axis=1)

# Drop the encoded columns showing NA or missing value
Births_Ethiopia_encoded = Births_Ethiopia_encoded.drop(['Residence_nan', 'Iron_8.0', 'Iron_nan', 'Anemia_nan', 'Occupation_96.0', 'Ethnicity_6.0', 'Ethnicity_7.0', 'Ethnicity_8.0',
       'Ethnicity_9.0', 'Ethnicity_10.0', 'Ethnicity_11.0', 'Ethnicity_12.0',
       'Ethnicity_13.0', 'Ethnicity_14.0', 'Ethnicity_15.0', 'Ethnicity_16.0',
       'Ethnicity_17.0', 'Ethnicity_18.0', 'Ethnicity_19.0', 'Ethnicity_20.0',
       'Ethnicity_22.0', 'Ethnicity_24.0', 'Ethnicity_28.0', 'Ethnicity_33.0',
       'Ethnicity_34.0', 'Ethnicity_35.0', 'Ethnicity_36.0', 'Ethnicity_38.0',
       'Ethnicity_42.0', 'Ethnicity_43.0', 'Ethnicity_44.0', 'Ethnicity_47.0',
       'Ethnicity_53.0', 'Ethnicity_57.0', 'Ethnicity_59.0', 'Ethnicity_64.0',
       'Ethnicity_71.0', 'Ethnicity_72.0', 'Ethnicity_73.0', 'Ethnicity_75.0',
       'Ethnicity_76.0', 'Ethnicity_85.0', 'Ethnicity_93.0', 'Ethnicity_96.0',
       'Ethnicity_99.0', 'Religion_96.0'], axis=1)
# Births_Ethiopia_encoded.head()

In [ ]:
# Numerical variables: imputation
def addlabels(x, y):
    for i in range(len(x)):
        plt.text(i,y[i],round(y[i],2))
        
missing_values = Births_Ethiopia_encoded.isnull().sum() / len(Births_Ethiopia_encoded)
missing_values = missing_values[missing_values > 0]
missing_values.sort_values(inplace=True)
missing_values = missing_values.to_frame()
missing_values.columns = ['Proportion of NaN']
missing_values.index.names = ['Name']
missing_values['Name'] = missing_values.index

sns.set(style="whitegrid", color_codes=True)
sns.barplot(x = 'Name', y = 'Proportion of NaN', data=missing_values)
plt.xticks(rotation = 45)
addlabels(missing_values['Name'], y=missing_values['Proportion of NaN'])
plt.show()

In [ ]:
# Impute with mean simpleImputer
imputer_sim = SimpleImputer(strategy = "mean")
imputed_df_sim = imputer_sim.fit_transform(Births_Ethiopia_encoded)
Births_Ethiopia_imputed = pd.DataFrame(imputed_df_sim)
Births_Ethiopia_imputed.columns = Births_Ethiopia_encoded.columns
# Births_Ethiopia_imputed.head()

In [ ]:
# Response variable engineering
Births_Ethiopia_imputed["Is_LBW"] = Births_Ethiopia_imputed["Birth_Weight"]<2500
Births_Ethiopia_res = Births_Ethiopia_imputed.drop(['Birth_Weight'], axis=1)
# Births_Ethiopia_res.head()


In [ ]:
Y_Ethiopia = Births_Ethiopia_res["Is_LBW"]
Feature_Ethiopia = Births_Ethiopia_res.drop(["Is_LBW"], axis=1)


In [ ]:
# Lasso for feature selection
lasso = LogisticRegression(penalty='l1', solver='liblinear', C=1.0, random_state=42)
lasso.fit(Feature_Ethiopia, Y_Ethiopia)

coefficients = lasso.coef_[0]
# Identify columns that were dropped (coefficients are zero)
selected_columns = Feature_Ethiopia.columns[coefficients != 0]
# print("Selected columns:", selected_columns)

# Select features based on the weights after Lasso regularization
feature_model = SelectFromModel(lasso, prefit=True)

# Transform the dataset to only include the selected features
Feature_new = feature_model.transform(Feature_Ethiopia)

# Update Feature df
Feature_Ethiopia = pd.DataFrame(Feature_new, columns = ['BMI', 'Age_W', 'Visits', 'Education_0.0', 'Education_2.0',
       'Education_3.0', 'Iron_0.0', 'Iron_1.0', 'Wealth_1.0', 'Wealth_2.0',
       'Wealth_4.0', 'Wealth_5.0', 'Anemia_3.0', 'Anemia_4.0', 'Smoking_0.0',
       'Insurance_0.0', 'Occupation_1.0', 'Occupation_2.0', 'Occupation_5.0',
       'Occupation_8.0', 'Occupation_9.0', 'Ethnicity_1.0', 'Ethnicity_2.0',
       'Ethnicity_3.0', 'Ethnicity_4.0', 'Ethnicity_5.0', 'Marital_0.0',
       'Marital_1.0', 'Marital_2.0', 'Marital_4.0', 'Marital_5.0',
       'Religion_1.0', 'Religion_4.0', 'Gender_1.0', 'Twin_0.0', 'Twin_1.0',
       'Delivery_Caesarean_1.0'])
# Feature_Ethiopia.head()

In [ ]:
# Whole Ethiopia df after data pre-processing
Ethiopia_final = pd.concat([Feature_Ethiopia, Y_Ethiopia], 1)
Ethiopia_final.head()

## Model construction and Evaluation (baseline models)

### Data split

In [ ]:
# Split training and testing set
X_train_Ethiopia, X_test_Ethiopia, Y_train_Ethiopia, Y_test_Ethiopia = train_test_split(Feature_Ethiopia, Y_Ethiopia, test_size=0.25, random_state=42)


In [ ]:
# Scale X if needed
scaler = StandardScaler()
X_train_Ethiopia_scaled = scaler.fit_transform(X_train_Ethiopia)
X_test_Ethiopia_scaled = scaler.transform(X_test_Ethiopia)

In [ ]:
# Get confidence interval for auc score
def roc_auc_ci(y_true, y_score, AUC, positive=True):
    N1 = sum(y_true)
    N2 = len(y_true) - N1
    Q1 = AUC / (2 - AUC)
    Q2 = 2*AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC*(1 - AUC) + (N1 - 1)*(Q1 - AUC**2) + (N2 - 1)*(Q2 - AUC**2)) / (N1*N2))
    lower = AUC - 1.96*SE_AUC
    upper = AUC + 1.96*SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower, upper)

### Logistic regression

In [ ]:
lr_model_Ethiopia = LogisticRegression(solver='liblinear', random_state=42)
lr_model_Ethiopia.fit(X_train_Ethiopia, Y_train_Ethiopia)
lr_predict_Ethiopia = lr_model_Ethiopia.predict(X_test_Ethiopia)
lr_score_Ethiopia = lr_model_Ethiopia.predict_proba(X_test_Ethiopia)[:,1]
lr_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, lr_score_Ethiopia)
print('roc_auc_score for Logistic Regression in Ethiopic data (without resampling): ', lr_auc_Ethiopia)
print('Confidence interval for AUC in Logistic Regression is: ', roc_auc_ci(Y_test_Ethiopia, lr_score_Ethiopia, lr_auc_Ethiopia))

### Decision tree

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_param_grid = {'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]}
dt_grid_search = GridSearchCV(estimator=dt_model, param_grid=dt_param_grid, cv=5, scoring='roc_auc')
dt_grid_search.fit(X_train_Ethiopia, Y_train_Ethiopia)
dt_best_params = dt_grid_search.best_params_
print(dt_best_params)


In [ ]:
dt_model_Ethiopia = dt_grid_search.best_estimator_
dt_model_Ethiopia.fit(X_train_Ethiopia, Y_train_Ethiopia)
dt_predict_Ethiopia = dt_model_Ethiopia.predict(X_test_Ethiopia)
dt_score_Ethiopia = dt_model_Ethiopia.predict_proba(X_test_Ethiopia)[:,1]
dt_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, dt_score_Ethiopia)
print('roc_auc_score for Decision Tree in Ethiopic data (without resampling): ', dt_auc_Ethiopia)
print('Confidence interval for AUC in Decision Tree is: ', roc_auc_ci(Y_test_Ethiopia, dt_score_Ethiopia, dt_auc_Ethiopia))

### Linear SVM

In [ ]:
svm_model_Ethiopia = SVC(kernel='linear', probability=True)
svm_model_Ethiopia.fit(X_train_Ethiopia, Y_train_Ethiopia)
svm_predict_Ethiopia = svm_model_Ethiopia.predict(X_test_Ethiopia)
svm_score_Ethiopia = svm_model_Ethiopia.predict_proba(X_test_Ethiopia)[:,1]
svm_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, svm_score_Ethiopia)
print('roc_auc_score for Linear Support Vectore Machine in Ethiopic data (without resampling): ', svm_auc_Ethiopia)
print('Confidence interval for AUC in Linear SVM is: ', roc_auc_ci(Y_test_Ethiopia, svm_score_Ethiopia, svm_auc_Ethiopia))

### KNN

In [ ]:
knn_model = KNeighborsClassifier()
knn_param_grid = {'n_neighbors': [10, 15, 20, 30, 40, 50, 60, 80, 100]}
knn_grid_search = GridSearchCV(knn_model, knn_param_grid, cv=5, scoring='roc_auc')
knn_grid_search.fit(X_train_Ethiopia, Y_train_Ethiopia)
knn_best_params = knn_grid_search.best_params_
print(knn_best_params)

In [ ]:
knn_model_Ethiopia = knn_grid_search.best_estimator_
knn_model_Ethiopia.fit(X_train_Ethiopia, Y_train_Ethiopia)
knn_predict_Ethiopia = knn_model_Ethiopia.predict(X_test_Ethiopia)
knn_score_Ethiopia = knn_model_Ethiopia.predict_proba(X_test_Ethiopia)[:,1]
knn_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, knn_score_Ethiopia)
print('roc_auc_score for K-nearest Neighbors in Ethiopic data (without resampling): ', knn_auc_Ethiopia)
print('Confidence interval for AUC in KNN is: ', roc_auc_ci(Y_test_Ethiopia, knn_score_Ethiopia, knn_auc_Ethiopia))

### Random Forest

In [ ]:
rf_model = RandomForestClassifier(random_state=42)
rf_param_rand = {'n_estimators': range(2, 20), 'max_depth': range(2,15)}
rf_rand_search = RandomizedSearchCV(rf_model, param_distributions = rf_param_rand, n_iter=5, cv=5,random_state = 42)
rf_rand_search.fit(X_train_Ethiopia, Y_train_Ethiopia)
rf_best_params = rf_rand_search.best_params_
print(rf_best_params)

In [ ]:
rf_model_Ethiopia = rf_rand_search.best_estimator_
rf_model_Ethiopia.fit(X_train_Ethiopia, Y_train_Ethiopia)
rf_predict_Ethiopia = rf_model_Ethiopia.predict(X_test_Ethiopia)
rf_score_Ethiopia = rf_model_Ethiopia.predict_proba(X_test_Ethiopia)[:,1]
rf_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, rf_score_Ethiopia)
print('roc_auc_score for Random Forest in Ethiopic data (without resampling): ', rf_auc_Ethiopia)
print('Confidence interval for AUC in Random Forest is: ', roc_auc_ci(Y_test_Ethiopia, rf_score_Ethiopia, rf_auc_Ethiopia))

### Adaboosting

In [ ]:
ada_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=42))
ada_param_grid = {'base_estimator__max_depth': [1, 2], 'n_estimators': range(100, 275, 25), 'learning_rate': [0.01, 0.02, 0.05]}
ada_grid_search = GridSearchCV(ada_model, ada_param_grid, cv=5, scoring='roc_auc')
ada_grid_search.fit(X_train_Ethiopia, Y_train_Ethiopia)
ada_best_params = ada_grid_search.best_params_
print(ada_best_params)

In [ ]:
ada_model_Ethiopia = ada_grid_search.best_estimator_
ada_model_Ethiopia.fit(X_train_Ethiopia, Y_train_Ethiopia)
ada_predict_Ethiopia = ada_model_Ethiopia.predict(X_test_Ethiopia)
ada_score_Ethiopia = ada_model_Ethiopia.predict_proba(X_test_Ethiopia)[:,1]
ada_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, ada_score_Ethiopia)
print('roc_auc_score for Adaboosting in Ethiopic data (without resampling): ', ada_auc_Ethiopia)
print('Confidence interval for AUC in Adaboosting is: ', roc_auc_ci(Y_test_Ethiopia, ada_score_Ethiopia, ada_auc_Ethiopia))

### Bagging

In [ ]:
bag_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=42), random_state=42)
bag_param_grid = {'n_estimators': range(100, 275, 25),
                  'base_estimator__max_depth': [3, 4, 5, 6],
                  'max_samples': [0.3, 0.5, 0.7, 1.0],
                  'max_features': [0.3, 0.5, 0.7, 1.0]}
bag_grid_search = GridSearchCV(bag_model, bag_param_grid, cv=5, scoring='roc_auc')
bag_grid_search.fit(X_train_Ethiopia_scaled, Y_train_Ethiopia)
bag_best_params = bag_grid_search.best_params_
print(bag_best_params)

In [ ]:
bag_model_Ethiopia = bag_grid_search.best_estimator_
bag_model_Ethiopia.fit(X_train_Ethiopia_scaled, Y_train_Ethiopia)
bag_predict_Ethiopia = bag_model_Ethiopia.predict(X_test_Ethiopia_scaled)
bag_score_Ethiopia = bag_model_Ethiopia.predict_proba(X_test_Ethiopia_scaled)[:,1]
bag_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, bag_score_Ethiopia)
print('roc_auc_score for Bagging in Ethiopic data (without resampling): ', bag_auc_Ethiopia)
print('Confidence interval for AUC in Bagging is: ', roc_auc_ci(Y_test_Ethiopia, bag_score_Ethiopia, bag_auc_Ethiopia))

### Gradient boosting

In [ ]:
gb_model = GradientBoostingClassifier(random_state=42)
gb_param_grid = {'n_estimators': range(150, 325, 25), 'learning_rate': [0.01, 0.02, 0.05, 0.1], 'max_depth': [3, 4, 5]}
gb_grid_search = GridSearchCV(gb_model, gb_param_grid, cv=5, scoring='roc_auc')
gb_grid_search.fit(X_train_Ethiopia_scaled, Y_train_Ethiopia)
gb_best_params = gb_grid_search.best_params_
print(gb_best_params)

In [ ]:
gb_model_Ethiopia = gb_grid_search.best_estimator_
gb_model_Ethiopia.fit(X_train_Ethiopia_scaled, Y_train_Ethiopia)
gb_predict_Ethiopia = gb_model_Ethiopia.predict(X_test_Ethiopia_scaled)
gb_score_Ethiopia = gb_model_Ethiopia.predict_proba(X_test_Ethiopia_scaled)[:,1]
gb_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, gb_score_Ethiopia)
print('roc_auc_score for Gradient Boosting in Ethiopic data (without resampling): ', gb_auc_Ethiopia)
print('Confidence interval for AUC in Gradient Boosting is: ', roc_auc_ci(Y_test_Ethiopia, gb_score_Ethiopia, gb_auc_Ethiopia))

## Resampling with SMOTE

In [ ]:
sampling_strategies = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

classifiers = {
    'LogisticRegression': lr_model_Ethiopia,
    'DecisionTree': dt_model_Ethiopia,
    'SVM': svm_model_Ethiopia,
    'KNN': knn_model_Ethiopia,
    'RandomForest': rf_model_Ethiopia,
    'Adaboosting': ada_model_Ethiopia,
    'Bagging': bag_model_Ethiopia,
    'GradientBoosting': gb_model_Ethiopia
}

for strategy in sampling_strategies:
    print(f"SMOTE Sampling Strategy: {strategy}")
    
    # Apply SMOTE with the current sampling strategy
    smote = SMOTE(sampling_strategy=strategy, random_state=42)
    X_train_smote, Y_train_smote = smote.fit_resample(X_train_Ethiopia, Y_train_Ethiopia)
    X_train_smote_scaled, Y_train_smote_scaled = smote.fit_resample(X_train_Ethiopia_scaled, Y_train_Ethiopia)
    
    # Iterate over classifiers
    for name, clf in classifiers.items():
        if name=='GradientBoosting' or name=='Bagging':
            clf.fit(X_train_smote_scaled, Y_train_smote_scaled)
            y_pred_proba = clf.predict_proba(X_test_Ethiopia_scaled)[:, 1]
        else:
            clf.fit(X_train_smote, Y_train_smote)
            y_pred_proba = clf.predict_proba(X_test_Ethiopia)[:, 1]

        auc_score = roc_auc_score(Y_test_Ethiopia, y_pred_proba)
        print(f"{name} AUC: {auc_score}")

In [ ]:
sampling_strategies = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
logistic_regression = [0.7418963231703063, 0.7292123072629597, 0.7343682411218866, 0.7383520546989465, 0.7471220260936301, 0.7494871973766832, 0.7519221377241331, 0.7533105421056304, 0.7527244819646969, 0.7529686736900859, 0.7536105490825368]
decision_tree = [0.7374973836600852, 0.7451405846647596, 0.748430196051071, 0.748430196051071, 0.748430196051071, 0.7326344798716249, 0.7517512035163608, 0.7517512035163608, 0.7517512035163608, 0.7517512035163608, 0.7517512035163608]
linear_svm = [0.6075978511128165, 0.68318565548036, 0.7200586060140932, 0.7369287657852508, 0.7424126142468429, 0.7432917044582433, 0.7402462847973209, 0.7448580199539524, 0.7443173097048769, 0.744135910137445, 0.7459847903439615]
knn = [0.5203865206167585, 0.5223295890602108, 0.5254273355194307, 0.5326798297634829, 0.5426044791739343, 0.5320833042628899, 0.540971883067048, 0.5436893881253052, 0.5415893392869602, 0.5418230656526896, 0.5437033419381846]
random_forest = [0.759422312146794, 0.7631723993581245, 0.7540605595478964, 0.7594118467871346, 0.7550338379962325, 0.7564710807228076, 0.7513256122235401, 0.7496651084908952, 0.7561745621991209, 0.7500697690643967, 0.7587106676899462]
adaboosting = [0.7732261215377102, 0.7697516221307471, 0.7677911114211959, 0.7691934696155724, 0.7700097676690155, 0.7682027489011372, 0.7711190957929255, 0.7706760622340054, 0.7699190678852997, 0.7713039838135771, 0.7712725877345985]
bagging = [0.762101444219633, 0.772081908881602, 0.7703830321635387, 0.7740563734040327, 0.774513360775832, 0.7743145189423011, 0.7727307611804926, 0.7718202748901137, 0.7712516570152795, 0.77367264354985, 0.7755808274611038]
gradient_boosting = [0.7689667201562828, 0.7705051280262332, 0.7692946347589479, 0.7680457685062443, 0.7684922905183842, 0.7727307611804926, 0.7747366217819018, 0.7712481685620596, 0.7736586897369707, 0.7750575594781275, 0.7748482522849369]

plt.figure(figsize=(10, 6))
plt.plot(sampling_strategies, logistic_regression, label='Logistic Regression', marker='o')
plt.plot(sampling_strategies, decision_tree, label='Decision Tree', marker='o')
plt.plot(sampling_strategies, linear_svm, label='Linear SVM', marker='o')
plt.plot(sampling_strategies, knn, label='KNN', marker='o')
plt.plot(sampling_strategies, random_forest, label='Random Forest', marker='o')
plt.plot(sampling_strategies, adaboosting, label='Adaboosting', marker='o')
plt.plot(sampling_strategies, bagging, label='Bagging', marker='o')
plt.plot(sampling_strategies, gradient_boosting, label='Gradient Boosting', marker='o')

plt.xlabel('Sampling Strategy')
plt.ylabel('AUC Score')
plt.title('AUC vs. Sampling Strategy')
plt.legend(fontsize=12)
plt.grid(True)
plt.savefig('Ethiopia_sampling.png', dpi=200, bbox_inches='tight', pad_inches=0)


### Best AUC & SMOTE strategy for each model

logistic_regression: 0.7536105490825368, SMOTE=1

decision_tree: 0.7517512035163608, SMOTE=1

linear_svm: 0.7459847903439615, SMOTE=1

knn: 0.5437033419381846, SMOTE=1

random_forest: 0.7631723993581245, SMOTE=0.1

adaboosting: 0.7732261215377102, SMOTE=0

bagging: 0.7755808274611038, SMOTE=1

gradient_boosting: 0.7750575594781275, SMOTE=0.9

Best model: bagging: 0.7755808274611038, SMOTE=1

### Draw AUC plots with SMOTE strategy

In [ ]:
# Initialize SMOTE
smote_1 = SMOTE(sampling_strategy=1, random_state=42)
X_train_Ethiopia_smote_1, Y_train_Ethiopia_smote_1 = smote_1.fit_resample(X_train_Ethiopia, Y_train_Ethiopia)
X_train_Ethiopia_smote_1_scaled, Y_train_Ethiopia_smote_1_scaled = smote_1.fit_resample(X_train_Ethiopia_scaled, Y_train_Ethiopia)

smote_01 = SMOTE(sampling_strategy=0.1, random_state=42)
X_train_Ethiopia_smote_01, Y_train_Ethiopia_smote_01 = smote_01.fit_resample(X_train_Ethiopia, Y_train_Ethiopia)

smote_06 = SMOTE(sampling_strategy=0.6, random_state=42)
X_train_Ethiopia_smote_06, Y_train_Ethiopia_smote_06 = smote_06.fit_resample(X_train_Ethiopia, Y_train_Ethiopia)

smote_09 = SMOTE(sampling_strategy=0.9, random_state=42)
X_train_Ethiopia_smote_09, Y_train_Ethiopia_smote_09 = smote_09.fit_resample(X_train_Ethiopia, Y_train_Ethiopia)
X_train_Ethiopia_smote_09_scaled, Y_train_Ethiopia_smote_09_scaled = smote_09.fit_resample(X_train_Ethiopia_scaled, Y_train_Ethiopia)


### Logistic Regression, SMOTE=1

In [ ]:
lr_model_Ethiopia = LogisticRegression(solver='liblinear', random_state=42)
lr_model_Ethiopia.fit(X_train_Ethiopia_smote_1, Y_train_Ethiopia_smote_1)
lr_predict_Ethiopia = lr_model_Ethiopia.predict(X_test_Ethiopia)
lr_score_Ethiopia = lr_model_Ethiopia.predict_proba(X_test_Ethiopia)[:,1]
lr_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, lr_score_Ethiopia)
print('roc_auc_score for Logistic Regression in Ethiopic data: ', lr_auc_Ethiopia)
print('Confidence interval for AUC in Logistic Regression is: ', roc_auc_ci(Y_test_Ethiopia, lr_score_Ethiopia, lr_auc_Ethiopia))


In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Ethiopia, lr_score_Ethiopia)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC Curve in Ethiopia')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('lr_roc_E.jpg')

### Decision Tree, SMOTE=0.6

In [ ]:
dt_model_Ethiopia = DecisionTreeClassifier(max_depth=3, random_state=42)
dt_model_Ethiopia.fit(X_train_Ethiopia_smote_06, Y_train_Ethiopia_smote_06)
dt_predict_Ethiopia = dt_model_Ethiopia.predict(X_test_Ethiopia)
dt_score_Ethiopia = dt_model_Ethiopia.predict_proba(X_test_Ethiopia)[:,1]
dt_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, dt_score_Ethiopia)
print('roc_auc_score for Decision Tree in Ethiopic data: ', dt_auc_Ethiopia)
print('Confidence interval for AUC in Decision Tree is: ', roc_auc_ci(Y_test_Ethiopia, dt_score_Ethiopia, dt_auc_Ethiopia))

In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Ethiopia, dt_score_Ethiopia)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree ROC Curve in Ethiopia')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('dt_roc_E.jpg')

### Linear SVM, SMOTE=1

In [ ]:
svm_model_Ethiopia = SVC(kernel='linear', probability=True)
svm_model_Ethiopia.fit(X_train_Ethiopia_smote_1, Y_train_Ethiopia_smote_1)
svm_predict_Ethiopia = svm_model_Ethiopia.predict(X_test_Ethiopia)
svm_score_Ethiopia = svm_model_Ethiopia.predict_proba(X_test_Ethiopia)[:,1]
svm_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, svm_score_Ethiopia)
print('roc_auc_score for Linear Support Vectore Machine in Ethiopic data: ', svm_auc_Ethiopia)
print('Confidence interval for AUC in Linear SVM is: ', roc_auc_ci(Y_test_Ethiopia, svm_score_Ethiopia, svm_auc_Ethiopia))

In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Ethiopia, svm_score_Ethiopia)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Linear SVM ROC Curve in Ethiopia')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('svm_roc_E.jpg')

### KNN, SMOTE=1

In [ ]:
knn_model_Ethiopia = knn_model = KNeighborsClassifier(n_neighbors=80)
knn_model_Ethiopia.fit(X_train_Ethiopia_smote_1, Y_train_Ethiopia_smote_1)
knn_predict_Ethiopia = knn_model_Ethiopia.predict(X_test_Ethiopia)
knn_score_Ethiopia = knn_model_Ethiopia.predict_proba(X_test_Ethiopia)[:,1]
knn_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, knn_score_Ethiopia)
print('roc_auc_score for K-nearest Neighbors in Ethiopic data: ', knn_auc_Ethiopia)
print('Confidence interval for AUC in KNN is: ', roc_auc_ci(Y_test_Ethiopia, knn_score_Ethiopia, knn_auc_Ethiopia))

In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Ethiopia, knn_score_Ethiopia)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('KNN ROC Curve in Ethiopia')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('knn_roc_E.jpg')

### Random Forest, SMOTE=0.1

In [ ]:
rf_model_Ethiopia = RandomForestClassifier(n_estimators=17, max_depth=5, random_state=42)
rf_model_Ethiopia.fit(X_train_Ethiopia_smote_01, Y_train_Ethiopia_smote_01)
rf_predict_Ethiopia = rf_model_Ethiopia.predict(X_test_Ethiopia)
rf_score_Ethiopia = rf_model_Ethiopia.predict_proba(X_test_Ethiopia)[:,1]
rf_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, rf_score_Ethiopia)
print('roc_auc_score for Random Forest in Ethiopic data: ', rf_auc_Ethiopia)
print('Confidence interval for AUC in Random Forest is: ', roc_auc_ci(Y_test_Ethiopia, rf_score_Ethiopia, rf_auc_Ethiopia))

In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Ethiopia, rf_score_Ethiopia)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Random Forest ROC Curve in Ethiopia')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('rf_roc_E.jpg')

### Adaboosting, SMOTE=0

In [ ]:
ada_model_Ethiopia = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, random_state=42), learning_rate=0.05, n_estimators=150, random_state=42)
ada_model_Ethiopia.fit(X_train_Ethiopia, Y_train_Ethiopia)
ada_predict_Ethiopia = ada_model_Ethiopia.predict(X_test_Ethiopia)
ada_score_Ethiopia = ada_model_Ethiopia.predict_proba(X_test_Ethiopia)[:,1]
ada_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, ada_score_Ethiopia)
print('roc_auc_score for Adaboosting in Ethiopic data: ', ada_auc_Ethiopia)
print('Confidence interval for AUC in Adaboosting is: ', roc_auc_ci(Y_test_Ethiopia, ada_score_Ethiopia, ada_auc_Ethiopia))


In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Ethiopia, ada_score_Ethiopia)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Adaboosting ROC Curve in Ethiopia')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('ada_roc_E.jpg')

### Bagging, SMOTE=1

In [ ]:
bag_model_Ethiopia = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=3, random_state=42), max_features=0.7, max_samples=1.0, n_estimators=100, random_state=42)
bag_model_Ethiopia.fit(X_train_Ethiopia_smote_1_scaled, Y_train_Ethiopia_smote_1_scaled)
bag_predict_Ethiopia = bag_model_Ethiopia.predict(X_test_Ethiopia_scaled)
bag_score_Ethiopia = bag_model_Ethiopia.predict_proba(X_test_Ethiopia_scaled)[:,1]
bag_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, bag_score_Ethiopia)
print('roc_auc_score for Bagging in Guinean data (without resampling): ', bag_auc_Ethiopia)
print('Confidence interval for AUC in Bagging is: ', roc_auc_ci(Y_test_Ethiopia, bag_score_Ethiopia, bag_auc_Ethiopia))


In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Ethiopia, bag_score_Ethiopia)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Bagging ROC Curve in Ethiopia')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('bag_roc_E.jpg')

### Gradient Boosting, SMOTE=0.9

In [ ]:
gb_model_Ethiopia = GradientBoostingClassifier(learning_rate=0.01, max_depth=3, n_estimators=275, random_state=42)
gb_model_Ethiopia.fit(X_train_Ethiopia_smote_09_scaled, Y_train_Ethiopia_smote_09_scaled)
gb_predict_Ethiopia = gb_model_Ethiopia.predict(X_test_Ethiopia_scaled)
gb_score_Ethiopia = gb_model_Ethiopia.predict_proba(X_test_Ethiopia_scaled)[:,1]
gb_auc_Ethiopia = roc_auc_score(Y_test_Ethiopia, gb_score_Ethiopia)
print('roc_auc_score for Gradient Boosting in Ethiopic data: ', gb_auc_Ethiopia)
print('Confidence interval for AUC in Gradient Boosting is: ', roc_auc_ci(Y_test_Ethiopia, gb_score_Ethiopia, gb_auc_Ethiopia))

In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Ethiopia, gb_score_Ethiopia)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Gradient Boosting ROC Curve in Ethiopia')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('gb_roc_E.jpg')

## OOD generalization

Do same steps for data from Guinea

In [ ]:
# Data from Guinea
Births_Guinea = pd.read_sas('Births_Guinea.SAS7BDAT')
print(Births_Guinea.shape)

# 'M19': Birth_Weight: Birth weight in kilograms (3 decimals) [Original Response variable]
print("The number of valid samples in the dataset is " + str(len(Births_Guinea['M19'].dropna())))
                                                            
Births_Guinea.head()

In [ ]:
Births_Guinea_df = Births_Guinea[['M19','V026','V106','H42','V190','V445','V447A','V457','V463A','V481','V717','V131',
                  'V501','V130','B4','B0','M14','M17']]
Births_Guinea_df.columns = ['Birth_Weight','Residence','Education','Iron','Wealth','BMI','Age_W','Anemia',
                     'Smoking','Insurance','Occupation','Ethnicity','Marital','Religion','Gender',
                    'Twin','Visits','Delivery_Caesarean']
Births_Guinea_df = Births_Guinea_df.dropna(subset=['Birth_Weight'])
Births_Guinea_df = Births_Guinea_df.reset_index()
Births_Guinea_df = Births_Guinea_df.iloc[:,1:]
print(Births_Guinea_df.shape)
Births_Guinea_df.head()

In [ ]:
# Categorical variables: one-hot encoding
enc = OneHotEncoder(sparse=False)
encoded_array = enc.fit_transform(Births_Guinea_df[['Residence', 'Education', 'Iron', 'Wealth', 'Anemia', 'Smoking', 'Insurance', 'Occupation', 'Ethnicity', 'Marital', 'Religion', 'Gender', 'Twin', 'Delivery_Caesarean']])

encoded_cols = enc.get_feature_names_out(['Residence', 'Education', 'Iron', 'Wealth', 'Anemia', 'Smoking', 'Insurance', 'Occupation', 'Ethnicity', 'Marital', 'Religion', 'Gender', 'Twin', 'Delivery_Caesarean'])
encoded_df = pd.DataFrame(encoded_array, columns=encoded_cols)

# Drop the original columns that were encoded & concatenate with the other columns
Births_Guinea_encoded = pd.concat([Births_Guinea_df.drop(['Residence', 'Education', 'Iron', 'Wealth', 'Anemia', 'Smoking', 'Insurance', 'Occupation', 'Ethnicity', 'Marital', 'Religion', 'Gender', 'Twin', 'Delivery_Caesarean'], axis=1), encoded_df], axis=1)

# Drop the encoded columns showing NA or missing value
Births_Guinea_encoded = Births_Guinea_encoded.drop(['Iron_8.0', 'Iron_nan', 'Anemia_nan', 'Occupation_10.0', 'Occupation_96.0', 'Occupation_98.0', 'Ethnicity_97.0', 'Delivery_Caesarean_nan'], axis=1)
# Births_Guinea_encoded.head()

In [ ]:
# Numerical variables: imputation
missing_values = Births_Guinea_encoded.isnull().sum() / len(Births_Guinea_encoded)
missing_values = missing_values[missing_values > 0]
missing_values.sort_values(inplace=True)
missing_values = missing_values.to_frame()
missing_values.columns = ['Proportion of NaN']
missing_values.index.names = ['Name']
missing_values['Name'] = missing_values.index

sns.set(style="whitegrid", color_codes=True)
sns.barplot(x = 'Name', y = 'Proportion of NaN', data=missing_values)
plt.xticks(rotation = 45)
addlabels(missing_values['Name'], y=missing_values['Proportion of NaN'])
plt.show()

In [ ]:
# Impute with mean simpleImputer
imputer_sim = SimpleImputer(strategy = "mean")
imputed_df_sim = imputer_sim.fit_transform(Births_Guinea_encoded)
Births_Guinea_imputed = pd.DataFrame(imputed_df_sim)
Births_Guinea_imputed.columns = Births_Guinea_encoded.columns
# Births_Guinea_imputed.head()

In [ ]:
# Response variable engineering
Births_Guinea_imputed["Is_LBW"] = Births_Guinea_imputed["Birth_Weight"]<2500
Births_Guinea_res = Births_Guinea_imputed.drop(['Birth_Weight'], axis=1)
# Births_Guinea_res.head()

In [ ]:
Y_Guinea = Births_Guinea_res["Is_LBW"]
Feature_Guinea = Births_Guinea_res.drop(["Is_LBW"], axis=1)

In [ ]:
# Lasso for feature selection
lasso = LogisticRegression(penalty='l1', solver='liblinear', C=1.0, random_state=42)
lasso.fit(Feature_Guinea, Y_Guinea)

coefficients = lasso.coef_[0]
# Identify columns that were dropped (coefficients are zero)
selected_columns = Feature_Guinea.columns[coefficients != 0]
# print("Selected columns:", selected_columns)

# Select features based on the weights after Lasso regularization
feature_model = SelectFromModel(lasso, prefit=True)

# Transform the dataset to only include the selected features
Feature_new = feature_model.transform(Feature_Guinea)

# Update Feature df
Feature_Guinea = pd.DataFrame(Feature_new, columns = ['BMI', 'Age_W', 'Visits', 'Residence_2.0', 'Education_1.0',
       'Education_2.0', 'Education_3.0', 'Iron_1.0', 'Wealth_1.0',
       'Wealth_2.0', 'Wealth_5.0', 'Anemia_1.0', 'Anemia_2.0', 'Anemia_3.0',
       'Anemia_4.0', 'Smoking_1.0', 'Insurance_0.0', 'Occupation_0.0',
       'Occupation_1.0', 'Occupation_2.0', 'Occupation_3.0', 'Occupation_4.0',
       'Occupation_7.0', 'Occupation_9.0', 'Ethnicity_1.0', 'Ethnicity_2.0',
       'Ethnicity_3.0', 'Ethnicity_4.0', 'Ethnicity_5.0', 'Ethnicity_6.0',
       'Marital_0.0', 'Marital_2.0', 'Marital_3.0', 'Marital_5.0',
       'Religion_1.0', 'Religion_2.0', 'Religion_4.0', 'Gender_1.0',
       'Twin_0.0', 'Delivery_Caesarean_0.0', 'Delivery_Caesarean_1.0'])
# Feature_Guinea.head()

In [ ]:
# Whole Guinea df after data pre-processing
Guinea_final = pd.concat([Feature_Guinea, Y_Guinea], 1)
Guinea_final.head()

In [ ]:
# Split training and testing set
X_train_Guinea, X_test_Guinea, Y_train_Guinea, Y_test_Guinea = train_test_split(Feature_Guinea, Y_Guinea, test_size=0.25, random_state=42)

In [ ]:
# Scale X if needed
scaler = StandardScaler()
X_train_Guinea_scaled = scaler.fit_transform(X_train_Guinea)
X_test_Guinea_scaled = scaler.transform(X_test_Guinea)

In [ ]:
# Initialize SMOTE
X_train_Guinea_smote_1, Y_train_Guinea_smote_1 = smote_1.fit_resample(X_train_Guinea, Y_train_Guinea)
X_train_Guinea_smote_1_scaled, Y_train_Guinea_smote_1_scaled = smote_1.fit_resample(X_train_Guinea_scaled, Y_train_Guinea)

X_train_Guinea_smote_01, Y_train_Guinea_smote_01 = smote_01.fit_resample(X_train_Guinea, Y_train_Guinea)

X_train_Guinea_smote_06, Y_train_Guinea_smote_06 = smote_06.fit_resample(X_train_Guinea, Y_train_Guinea)

X_train_Guinea_smote_09, Y_train_Guinea_smote_09 = smote_09.fit_resample(X_train_Guinea, Y_train_Guinea)
X_train_Guinea_smote_09_scaled, Y_train_Guinea_smote_09_scaled = smote_09.fit_resample(X_train_Guinea_scaled, Y_train_Guinea)


### Logistic Regression, SMOTE=1

In [ ]:
lr_model_Guinea = LogisticRegression(solver='liblinear', random_state=42)
lr_model_Guinea.fit(X_train_Guinea_smote_1, Y_train_Guinea_smote_1)
lr_predict_Guinea = lr_model_Guinea.predict(X_test_Guinea)
lr_score_Guinea = lr_model_Guinea.predict_proba(X_test_Guinea)[:,1]
lr_auc_Guinea = roc_auc_score(Y_test_Guinea, lr_score_Guinea)
print('roc_auc_score for Logistic Regression in Guinean data: ', lr_auc_Guinea)
print('Confidence interval for AUC in Logistic Regression is: ', roc_auc_ci(Y_test_Guinea, lr_score_Guinea, lr_auc_Guinea))


In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Guinea, lr_score_Guinea)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC Curve in Guinea')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('lr_roc_G.jpg')

### Decision Tree, SMOTE=1

In [ ]:
dt_model_Guinea = DecisionTreeClassifier(max_depth=3, random_state=42)
dt_model_Guinea.fit(X_train_Guinea_smote_1, Y_train_Guinea_smote_1)
dt_predict_Guinea = dt_model_Guinea.predict(X_test_Guinea)
dt_score_Guinea = dt_model_Guinea.predict_proba(X_test_Guinea)[:,1]
dt_auc_Guinea = roc_auc_score(Y_test_Guinea, dt_score_Guinea)
print('roc_auc_score for Decision Tree in Guinean data: ', dt_auc_Guinea)
print('Confidence interval for AUC in Decision Tree is: ', roc_auc_ci(Y_test_Guinea, dt_score_Guinea, dt_auc_Guinea))

In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Guinea, dt_score_Guinea)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree ROC Curve in Guinea')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('dt_roc_G.jpg')

### Linear SVM, SMOTE=1

In [ ]:
svm_model_Guinea = SVC(kernel='linear', probability=True)
svm_model_Guinea.fit(X_train_Guinea_smote_1, Y_train_Guinea_smote_1)
svm_predict_Guinea = svm_model_Guinea.predict(X_test_Guinea)
svm_score_Guinea = svm_model_Guinea.predict_proba(X_test_Guinea)[:,1]
svm_auc_Guinea = roc_auc_score(Y_test_Guinea, svm_score_Guinea)
print('roc_auc_score for Linear Support Vectore Machine in Guinean data: ', svm_auc_Guinea)
print('Confidence interval for AUC in SVM is: ', roc_auc_ci(Y_test_Guinea, svm_score_Guinea, svm_auc_Guinea))

In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Guinea, svm_score_Guinea)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Linear SVM ROC Curve in Guinea')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('svm_roc_G.jpg')

### KNN, SMOTE=1

In [ ]:
knn_model_Guinea = knn_model = KNeighborsClassifier(n_neighbors=80)
knn_model_Guinea.fit(X_train_Guinea_smote_1, Y_train_Guinea_smote_1)
knn_predict_Guinea = knn_model_Guinea.predict(X_test_Guinea)
knn_score_Guinea = knn_model_Guinea.predict_proba(X_test_Guinea)[:,1]
knn_auc_Guinea = roc_auc_score(Y_test_Guinea, knn_score_Guinea)
print('roc_auc_score for K-nearest Neighbors in Guinean data: ', knn_auc_Guinea)
print('Confidence interval for AUC in KNN is: ', roc_auc_ci(Y_test_Guinea, knn_score_Guinea, knn_auc_Guinea))

In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Guinea, knn_score_Guinea)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('KNN ROC Curve in Guinea')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('knn_roc_G.jpg')

### Random Forest, SMOTE=0.1

In [ ]:
rf_model_Guinea = RandomForestClassifier(n_estimators=17, max_depth=5, random_state=42)
rf_model_Guinea.fit(X_train_Guinea_smote_01, Y_train_Guinea_smote_01)
rf_predict_Guinea = rf_model_Guinea.predict(X_test_Guinea)
rf_score_Guinea = rf_model_Guinea.predict_proba(X_test_Guinea)[:,1]
rf_auc_Guinea = roc_auc_score(Y_test_Guinea, rf_score_Guinea)
print('roc_auc_score for Random Forest in Guinean data: ', rf_auc_Guinea)
print('Confidence interval for AUC in Random Forest is: ', roc_auc_ci(Y_test_Guinea, rf_score_Guinea, rf_auc_Guinea))

In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Guinea, rf_score_Guinea)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Random Forest ROC Curve in Guinea')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('rf_roc_G.jpg')

### Adaboosting, SMOTE=0

In [ ]:
ada_model_Guinea = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, random_state=42), learning_rate=0.05, n_estimators=150, random_state=42)
ada_model_Guinea.fit(X_train_Guinea, Y_train_Guinea)
ada_predict_Guinea = ada_model_Guinea.predict(X_test_Guinea)
ada_score_Guinea = ada_model_Guinea.predict_proba(X_test_Guinea)[:,1]
ada_auc_Guinea = roc_auc_score(Y_test_Guinea, ada_score_Guinea)
print('roc_auc_score for Adaboosting in Guinean data (without resampling): ', ada_auc_Guinea)
print('Confidence interval for AUC in Adaboosting is: ', roc_auc_ci(Y_test_Guinea, ada_score_Guinea, ada_auc_Guinea))


In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Guinea, ada_score_Guinea)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Adaboosting ROC Curve in Guinea')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('ada_roc_G.jpg')

### Bagging, SMOTE=1

In [ ]:
bag_model_Guinea = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=3, random_state=42), max_features=0.7, max_samples=1.0, n_estimators=100, random_state=42)
bag_model_Guinea.fit(X_train_Guinea_smote_1_scaled, Y_train_Guinea_smote_1_scaled)
bag_predict_Guinea = bag_model_Guinea.predict(X_test_Guinea_scaled)
bag_score_Guinea = bag_model_Guinea.predict_proba(X_test_Guinea_scaled)[:,1]
bag_auc_Guinea = roc_auc_score(Y_test_Guinea, bag_score_Guinea)
print('roc_auc_score for Bagging in Guinean data: ', bag_auc_Guinea)
print('Confidence interval for AUC in Bagging is: ', roc_auc_ci(Y_test_Guinea, bag_score_Guinea, bag_auc_Guinea))


In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Guinea, bag_score_Guinea)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Bagging ROC Curve in Guinea')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('bag_roc_G.jpg')

### Gradient Boosting, SMOTE=0.9

In [ ]:
gb_model_Guinea = GradientBoostingClassifier(learning_rate=0.01, max_depth=3, n_estimators=275, random_state=42)
gb_model_Guinea.fit(X_train_Guinea_smote_09_scaled, Y_train_Guinea_smote_09_scaled)
gb_predict_Guinea = gb_model_Guinea.predict(X_test_Guinea_scaled)
gb_score_Guinea = gb_model_Guinea.predict_proba(X_test_Guinea_scaled)[:,1]
gb_auc_Guinea = roc_auc_score(Y_test_Guinea, gb_score_Guinea)
print('roc_auc_score for Gradient Boosting in Guinean data: ', gb_auc_Guinea)
print('Confidence interval for AUC in Gradient Boosting is: ', roc_auc_ci(Y_test_Guinea, gb_score_Guinea, gb_auc_Guinea))

In [ ]:
# Draw ROC curve
fpr, tpr, thresholds = roc_curve(Y_test_Guinea, gb_score_Guinea)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Gradient Boosting ROC Curve in Guinea')
plt.legend(loc="lower right")

# Use seaborn for more beautiful aesthetics
sns.set_style("whitegrid")

# plt.show()
plt.savefig('gb_roc_G.jpg')

## Feature Imbalance Analysis

### Ethiopia

The best model is Bagging with SMOTE=1

In [ ]:
# Get feature importances
feature_importance_Ethiopia = np.mean([tree.feature_importances_ for tree in bag_model_Ethiopia.estimators_], axis=0)
feature_name_Ethiopia = [str(Feature_Ethiopia.columns[i]) for i in range(1, len(feature_importance_Ethiopia) + 1)]
feature_importance_Ethiopia = pd.Series(feature_importance_Ethiopia, index=feature_name_Ethiopia)

# Sort features by their importance
feature_importance_Ethiopia = feature_importance_Ethiopia.sort_values(ascending=False)
print(feature_importance_Ethiopia.head(10))

### Guinea

The best model is Random Forest with SMOTE=0.1

In [ ]:
# Get feature importances
feature_importance_Guinea = rf_model_Guinea.feature_importances_
feature_importance_Guinea = pd.Series(feature_importance_Guinea, index=Feature_Guinea.columns)

# Sort features by their importance
feature_importance_Guinea = feature_importance_Guinea.sort_values(ascending=False)
print(feature_importance_Guinea.head(10))